<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5d40bd1b64112905646f6b833510a6387aa922ad8c2601e362221996ac714ed6
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-04 11:46:38
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.45 C
-------------------
Today PnL: -68.56 K (-0.47%)
Current PnL: -19.89 L (-13.03%)
CY Booked + Current PnL: -7.24 L (-4.74%)
-------------------
Total profit:  2.15 L
Total loss:  -22.04 L
-------------------
Total Booked + Current PnL: 19.74 L (15.71%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.74%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 86.87 L (60.05%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.32%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.28,68.0,X-LC,5.54,232034.0,21668.0,7123.0,0.18,10.30,3.07,13.68,37.0,3.04,1.62,28.53,XY25,NTT,REFINERIES
78,TTKPRESTIG,770.00,104.06,54.0,M-SC,1.72,87953.0,-12824.0,12920.0,-1.08,-12.73,14.69,0.09,245.0,-0.99,0.61,15.55,OX40N,NTT,DURABLES
79,UNIONBANK,163.00,-5.43,66.0,M-LC,10.15,171346.0,30506.0,13982.0,-0.12,21.66,8.16,31.59,66.0,2.18,1.19,53.89,XY24,NTT,BANKS
33,ICICIGI,2252.93,-12.45,64.0,X-MC,5.24,150442.0,14309.0,16278.0,1.27,10.51,10.82,22.47,91.0,0.88,1.05,24.85,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-10.45,42.0,X-MC,2.49,88545.0,-2133.0,17036.0,1.65,-2.35,19.24,16.44,101.0,-0.13,0.62,17.35,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.0,-51.56,76.0,H-SC,6.75,275752.0,-10753.0,268086.0,1.84,-3.75,97.22,89.82,134.0,-0.04,1.92,45.89,XY24,NTT,FINANCE
74,TATAMOTORS,1065.0,-51.85,23.0,X-LC,26.39,151371.0,-125152.0,235231.0,1.71,-45.26,155.40,39.81,54.0,-0.53,1.05,6.69,XY24,NTT,AUTO
7,ASIANTILES,137.0,7516.67,71.0,L-SC,9.86,83572.0,-10238.0,86856.0,1.70,-10.91,103.93,81.67,269.0,-0.12,0.58,61.84,XR,NTT,CERAMICS
2,ABBOTINDIA,35195.0,-10.45,42.0,X-MC,2.49,88545.0,-2133.0,17036.0,1.65,-2.35,19.24,16.44,101.0,-0.13,0.62,17.35,X40,ATH,PHARMA
21,DABUR,735.0,-8.52,54.0,X-MC,3.09,247568.0,-500.0,108905.0,1.42,-0.20,43.99,43.70,102.0,-0.00,1.72,14.22,XY24,BTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TITAGARH,1548.0,-5.17,49.0,H-SC,3.62,202418.0,-39759.0,150518.0,-2.11,-16.42,74.36,45.74,158.0,-0.26,1.41,31.65,XY24,NTT,ENGINEERING
16,CAMS,4762.0,-6.05,45.0,X-SC,0.81,251909.0,53.0,62373.0,-2.07,0.02,24.76,24.79,122.0,0.00,1.75,22.70,X40N,NTT,MISC
3,ACC,3906.0,-38.50,44.0,X-MC,2.46,183300.0,-54651.0,207294.0,-2.00,-22.97,113.09,64.15,174.0,-0.26,1.28,2.82,XY24,BTT,CEMENT
55,RAJOOENG,143.1,-43.16,43.0,H-SC,17.04,86040.0,-16460.0,57062.0,-1.85,-16.06,66.32,39.61,114.0,-0.29,0.60,5.08,AR,ATH,MISC
56,RECLTD,446.0,45.36,49.0,M-LC,4.96,204710.0,1540.0,40594.0,-1.85,0.76,19.83,20.74,55.0,0.04,1.43,7.67,XY25,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,45.36,49.0,M-LC,4.96,204710.0,1540.0,40594.0,-1.85,0.76,19.83,20.74,55.0,0.04,1.43,7.67,XY25,NTT,FINANCE
31,HINDZINC,730.22,25.96,46.0,M-LC,10.68,205603.0,527.0,112033.0,-1.66,0.26,54.49,54.89,52.0,0.00,1.43,24.41,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,104.06,54.0,M-SC,1.72,87953.0,-12824.0,12920.0,-1.08,-12.73,14.69,0.09,245.0,-0.99,0.61,15.55,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,44.0,H-SC,2.62,220788.0,-48879.0,85202.0,-1.54,-18.13,38.59,13.47,138.0,-0.57,1.54,12.26,XY24,NTT,PAINTS
17,CERA,9475.0,-21.96,53.0,H-SC,1.97,142404.0,-33499.0,75517.0,-1.46,-19.04,53.03,23.89,149.0,-0.44,0.99,22.99,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-24.38,43.0,H-MC,6.04,102277.0,-29558.0,71778.0,-0.94,-22.42,70.18,32.03,98.0,-0.41,0.71,14.56,OX40N,NTT,IT
67,SIS,528.0,2019.69,44.0,H-SC,2.99,85552.0,-25480.0,49090.0,0.72,-22.95,57.38,21.26,156.0,-0.52,0.60,15.59,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,66.0,M-LC,10.15,171346.0,30506.0,13982.0,-0.12,21.66,8.16,31.59,66.0,2.18,1.19,53.89,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,49.96,60.0,M-MC,9.73,236930.0,11968.0,155687.0,-0.28,5.32,65.71,74.53,192.0,0.08,1.65,39.29,XY24,BTT,STEEL
56,RECLTD,446.00,45.36,49.0,M-LC,4.96,204710.0,1540.0,40594.0,-1.85,0.76,19.83,20.74,55.0,0.04,1.43,7.67,XY25,NTT,FINANCE
31,HINDZINC,730.22,25.96,46.0,M-LC,10.68,205603.0,527.0,112033.0,-1.66,0.26,54.49,54.89,52.0,0.00,1.43,24.41,X5K,ATH,METALS
46,KPIGREEN,731.05,13.63,75.0,H-SC,6.56,135513.0,10216.0,48717.0,-0.19,8.15,35.95,47.03,141.0,0.21,0.94,69.67,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,66.0,M-LC,10.15,171346.0,30506.0,13982.0,-0.12,21.66,8.16,31.59,66.0,2.18,1.19,53.89,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,13.63,75.0,H-SC,6.56,135513.0,10216.0,48717.0,-0.19,8.15,35.95,47.03,141.0,0.21,0.94,69.67,MH,ATH,POWER
85,WHIRLPOOL,2270.00,-39.15,58.0,M-SC,4.63,126104.0,11306.0,82737.0,-0.55,9.85,65.61,81.92,223.0,0.14,0.88,49.29,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,56.0,H-SC,6.07,130645.0,7309.0,124322.0,-0.07,5.93,95.16,106.72,119.0,0.06,0.91,30.69,AR,ATH,MISC
31,HINDZINC,730.22,25.96,46.0,M-LC,10.68,205603.0,527.0,112033.0,-1.66,0.26,54.49,54.89,52.0,0.00,1.43,24.41,X5K,ATH,METALS
86,WIPRO,420.00,-16.06,39.0,M-LC,5.41,147634.0,-3311.0,112763.0,-0.99,-2.19,76.38,72.51,53.0,-0.03,1.03,3.70,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-51.85,23.0,X-LC,26.39,151371.0,-125152.0,235231.0,1.71,-45.26,155.40,39.81,54.0,-0.53,1.05,6.69,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-40.72,27.0,X-SC,6.33,88627.0,-40043.0,85339.0,-0.25,-31.12,96.29,35.20,219.0,-0.47,0.62,1.72,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-22.12,32.0,X-MC,4.84,195581.0,-4766.0,35263.0,-0.07,-2.38,18.03,15.23,178.0,-0.14,1.36,29.10,X40N,NTT,REALTY
30,HINDUNILVR,2922.00,-12.79,35.0,X-LC,7.02,244990.0,-12073.0,47210.0,-0.41,-4.70,19.27,13.67,24.0,-0.26,1.71,14.75,XY25,NTT,FMCG
51,PAGEIND,51769.93,-28.87,36.0,X-MC,7.09,80320.0,-3190.0,23221.0,-1.47,-3.82,28.91,23.98,82.0,-0.14,0.56,1.78,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-37.92,50.0,X-LC,1.16,201101.0,963.0,19929.0,-0.65,0.48,9.91,10.44,4.0,0.05,1.40,6.07,X40,NTT,FMCG
66,SIEMENS,4671.50,-3.47,45.0,H-LC,1.22,155070.0,-31025.0,78512.0,-0.83,-16.67,50.63,25.51,15.0,-0.40,1.08,14.27,AR,ATH,ELECTRICAL
52,PGHH,17907.65,-30.55,39.0,X-MC,1.28,200985.0,165.0,67631.0,0.06,0.08,33.65,33.76,80.0,0.00,1.40,5.28,X40,ATH,FMCG
28,HAVELLS,2069.16,-6.32,45.0,X-MC,2.90,312690.0,-15301.0,121824.0,-0.55,-4.66,38.96,32.48,92.0,-0.13,2.18,6.23,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-4.81,74.0,X-LC,4.25,249555.0,15411.0,33216.0,0.23,6.58,13.31,20.77,86.0,0.46,1.74,13.96,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.76,38.0,X-SC,37.73,50928.0,-14078.0,165002.0,-1.22,-21.66,323.99,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-40.72,27.0,X-SC,6.33,88627.0,-40043.0,85339.0,-0.25,-31.12,96.29,35.20,219.0,-0.47,0.62,1.72,X40,NTT,FOOTWEAR
51,PAGEIND,51769.93,-28.87,36.0,X-MC,7.09,80320.0,-3190.0,23221.0,-1.47,-3.82,28.91,23.98,82.0,-0.14,0.56,1.78,X40,ATH,APPARELS
20,COLPAL,3726.84,-6.30,43.0,X-MC,7.36,218900.0,-44465.0,153777.0,-0.52,-16.88,70.25,41.51,84.0,-0.29,1.52,2.72,XY25,ATH,FMCG
3,ACC,3906.00,-38.50,44.0,X-MC,2.46,183300.0,-54651.0,207294.0,-2.00,-22.97,113.09,64.15,174.0,-0.26,1.28,2.82,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.76,38.0,X-SC,37.73,50928.0,-14078.0,165002.0,-1.22,-21.66,323.99,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-28.87,36.0,X-MC,7.09,80320.0,-3190.0,23221.0,-1.47,-3.82,28.91,23.98,82.0,-0.14,0.56,1.78,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.36,41.0,X-SC,4.46,81310.0,-63850.0,139780.0,-0.45,-43.99,171.91,52.31,136.0,-0.46,0.57,8.84,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.45,42.0,X-MC,2.49,88545.0,-2133.0,17036.0,1.65,-2.35,19.24,16.44,101.0,-0.13,0.62,17.35,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-40.72,27.0,X-SC,6.33,88627.0,-40043.0,85339.0,-0.25,-31.12,96.29,35.20,219.0,-0.47,0.62,1.72,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,-29.65,45.0,X-LC,10.09,280496.0,-65460.0,132170.0,-1.09,-18.92,47.12,19.28,1.0,-0.50,1.95,3.70,X40,ATH,IT
39,INFY,2275.00,-19.60,49.0,X-LC,5.25,313579.0,691.0,170995.0,-0.90,0.22,54.53,54.87,3.0,0.00,2.18,8.52,X40,BTT,IT
41,ITC,452.00,-37.92,50.0,X-LC,1.16,201101.0,963.0,19929.0,-0.65,0.48,9.91,10.44,4.0,0.05,1.40,6.07,X40,NTT,FMCG
83,VBL,671.64,-16.73,53.0,X-LC,7.21,299860.0,-15982.0,128640.0,-1.00,-5.06,42.90,35.67,5.0,-0.12,2.09,8.09,X40N,ATH,FMCG
1,ABB,7934.00,-39.87,51.0,H-LC,5.22,251160.0,-10459.0,129674.0,-0.45,-4.00,51.63,45.57,7.0,-0.08,1.75,6.87,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,42.0,L-SC,28.08,78298.0,-35251.0,75299.0,-1.16,-31.04,96.17,35.27,268.0,-0.47,0.55,91.64,XR,NTT,HOTELS
7,ASIANTILES,137.00,7516.67,71.0,L-SC,9.86,83572.0,-10238.0,86856.0,1.70,-10.91,103.93,81.67,269.0,-0.12,0.58,61.84,XR,NTT,CERAMICS
49,MASFIN,398.61,-16.89,57.0,H-SC,8.81,94395.0,-3585.0,25185.0,-1.73,-3.66,26.68,22.05,152.0,-0.14,0.66,37.21,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-39.15,58.0,M-SC,4.63,126104.0,11306.0,82737.0,-0.55,9.85,65.61,81.92,223.0,0.14,0.88,49.29,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,56.0,H-SC,6.07,130645.0,7309.0,124322.0,-0.07,5.93,95.16,106.72,119.0,0.06,0.91,30.69,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,80.72,73.0,H-SC,1.76,156905.0,-25870.0,136947.0,-1.08,-14.15,87.28,60.77,125.0,-0.19,1.09,40.53,XR,NTT,JEWELLERY
59,REPCOHOME,880.00,-51.56,76.0,H-SC,6.75,275752.0,-10753.0,268086.0,1.84,-3.75,97.22,89.82,134.0,-0.04,1.92,45.89,XY24,NTT,FINANCE
46,KPIGREEN,731.05,13.63,75.0,H-SC,6.56,135513.0,10216.0,48717.0,-0.19,8.15,35.95,47.03,141.0,0.21,0.94,69.67,MH,ATH,POWER
62,SAMMAANCAP,326.00,-198.86,75.0,M-SC,32.30,170091.0,20871.0,123316.0,-0.49,13.99,72.50,96.62,208.0,0.17,1.18,84.08,XY25,NTT,FINANCE
68,SONACOMS,806.63,-32.06,66.0,M-SC,7.25,84595.0,-16564.0,56569.0,0.45,-16.37,66.87,39.54,202.0,-0.29,0.59,19.29,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.61
1,25,44.06
2,50,75.45


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.09
LC    32.93
MC    22.95
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.00
X40      16.14
X40N     12.44
XY25     11.90
XR        9.93
AR        7.82
OX40N     6.54
X200      1.76
X5K       1.43
SR        1.07
MH        0.94
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.41
X-LC    22.29
X-MC    18.65
M-SC    12.70
X-SC     5.51
M-LC     5.08
H-LC     4.59
H-MC     2.34
M-MC     1.65
L-SC     1.47
L-LC     0.97
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.16,-3.88,37.46
IT,12.36,-18.43,81.87
FINANCE,12.23,-7.72,56.55
MISC,7.75,-16.54,75.32
ELECTRICAL,5.78,-8.86,47.46
PAINTS,5.46,-17.70,35.24
BANKS,4.04,-16.32,83.99
INSURANCE,3.86,0.17,37.05
AUTO,2.78,-43.83,103.86


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3263591.0,22
XR,1228363.0,13
AR,1164135.0,9
X40,840523.0,12
X40N,681168.0,9
OX40N,545733.0,9
XY25,463509.0,8
SR,267744.0,2
X5K,112033.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3324686.0,24
M-SC,1218391.0,15
X-MC,1195956.0,14
X-LC,1077413.0,13
X-SC,556586.0,6
H-LC,280045.0,3
M-LC,279372.0,4
L-SC,251245.0,3
H-MC,251044.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1167049.0      6
           AR         833173.0      5
           XR         765459.0      7
M-SC       XY24       699472.0      6
X-MC       XY24       576341.0      4
X-LC       X40        467834.0      5
           XY24       306792.0      2
X-SC       X40N       306245.0      4
X-MC       X40        287350.0      6
H-SC       SR         267744.0      2
           OX40N      242544.0      3
X-LC       X40N       221335.0      3
H-LC       AR         208186.0      2
H-MC       XY24       179266.0      1
X-MC       XY25       178677.0      2
X-SC       XY24       165002.0      1
L-SC       XR         162155.0      2
M-MC       XY24       155687.0      1
X-MC       X40N       153588.0      2
M-SC       OX40N      142321.0      4
           XR         130506.0      2
           XY25       123316.0      1
           AR         122776.0      2
M-LC       XR         112763.0      1
           X5K        112033.0      1
L-SC       OX40N       89090.0      1
X-SC       X40         85339.0      1
X-LC       XY25        81452.0      3
H-LC       X200        71859.0      1
H-MC       OX40N       71778.0      1
L-MC       XR          57480.0      1
H-SC       MH          48717.0      1
M-LC       XY25        40594.0      1
L-LC       XY25        39470.0      1
M-LC       XY24        13982.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 40.0 seconds
